### Dataset creation

In [170]:
import pandas as pd
import numpy as np
from sklearn.utils import shuffle

- load data and labels (normal and different attack types)

In [171]:
df_data = pd.read_csv("../datasets/thyroid/thyroid-labelled.csv")
df_data = shuffle(df_data)
df_labels = pd.DataFrame(df_data.iloc[:,-1])   
df_data = pd.DataFrame(df_data.iloc[:, :-1]).astype(float) 

- read column names and types

In [172]:
# col_names=[]
# col_datatypes=[]

# with open("datasets/kddcup/kddcup.names") as file:
#     next(file)    ### skip first line
#     for line in file:
#         name, datatype = line.split(": ")
#         col_names.append(name)
#         col_datatypes.append(datatype.replace(".\n",""))
        
# df_data.columns = col_names
df_labels.columns = ["labels"]

In [173]:
df_data.head()

,age,a,b,c,d,e,f,g,h,i,...,k,l,m,n,o,TSH,T3,TT4,T4U,FTI
4904,0.47,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00110,0.0230,0.113,0.110,0.103
1216,0.32,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00170,0.0170,0.101,0.080,0.126
3364,0.69,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00041,0.0201,0.117,0.114,0.103
3286,0.08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00929,0.0240,0.126,0.096,0.131
1724,0.61,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00006,0.0530,0.210,0.087,0.241


- check integrity of the data

In [174]:
print(df_data.isnull().values.any())

print(df_data.isna().values.any())

False
False


### Enumerate labels 

- ### normal samples are named as "anomaly" since they are minority (stated in paper)

In [175]:
# Considering n labelled data as normal and others as anomaly in thyroid dataset. assigning labels accordingly.
# Added by Jyotirmay.
# 1 = normal, 0 = anomaly
df_data["label"] = np.where(df_labels["labels"].str.contains("n"), 'normal', 'anomaly')
print(type(df_data.TSH[0]))

<class 'numpy.float64'>


In [176]:
### number of normal and abnormal
print("number of normal:", len(np.where(df_data["label"] == 'normal')[0]))
print("number of anomaly:", len(np.where(df_data["label"] == 'anomaly')[0]))

number of normal: 6603
number of anomaly: 250


In [177]:
## check in which location data is abnormal

np.where(df_data["label"] == 'normal')

(array([   0,    1,    2, ..., 6850, 6851, 6852]),)

In [178]:
### looks like working

print(df_data.iloc[740:750, -1])
print(df_labels.iloc[740:750, -1])
# df_data.head()

5474    normal
4216    normal
2610    normal
5516    normal
1915    normal
435     normal
1095    normal
4879    normal
728     normal
5289    normal
Name: label, dtype: object
5474    n
4216    n
2610    n
5516    n
1915    n
435     n
1095    n
4879    n
728     n
5289    n
Name: labels, dtype: object


In [179]:
### for each object column in data

cols_toOneHot = df_data.select_dtypes(include='object').columns.tolist()

cols_all = df_data.columns.tolist()
cols_toNormalize = list(set(cols_all) - set(cols_toOneHot))

In [180]:
print("columns to normalize:\n%s\n" % cols_toNormalize)
print("columns to one-hot:\n%s\n" % cols_toOneHot)

columns to normalize:
['e', 'k', 'm', 'c', 'a', 'n', 'd', 'l', 'FTI', 'o', 'T3', 'g', 'j', 'h', 'i', 'b', 'T4U', 'TSH', 'f', 'TT4', 'age']

columns to one-hot:
['label']



In [181]:
len(cols_toNormalize)

21

### Dataset preprocessing in numerical columns

In [182]:
print(df_data.TSH.dtype)

float64


In [183]:
def checkStats(df_data):
    i=0
    for col in df_data.columns.tolist():
            
        if col in cols_toOneHot:    ### skip if column is not numerical  ### TODO: check column type??
            i += 1
            continue

        min_ = df_data[col].values.min()
        max_ = df_data[col].values.max()
        std_ = df_data[col].values.std()

        print("column:%d min:%f max:%f std:%f" % (i, min_, max_, std_))
        i += 1

In [184]:
## before normalization
df_data = df_data
checkStats(df_data)

column:0 min:0.010000 max:0.970000 std:0.189447
column:1 min:0.000000 max:1.000000 std:0.461501
column:2 min:0.000000 max:1.000000 std:0.340448
column:3 min:0.000000 max:1.000000 std:0.123975
column:4 min:0.000000 max:1.000000 std:0.113844
column:5 min:0.000000 max:1.000000 std:0.192456
column:6 min:0.000000 max:1.000000 std:0.106077
column:7 min:0.000000 max:1.000000 std:0.118127
column:8 min:0.000000 max:1.000000 std:0.128450
column:9 min:0.000000 max:1.000000 std:0.242769
column:10 min:0.000000 max:1.000000 std:0.254232
column:11 min:0.000000 max:1.000000 std:0.113218
column:12 min:0.000000 max:1.000000 std:0.092386
column:13 min:0.000000 max:1.000000 std:0.157747
column:14 min:0.000000 max:1.000000 std:0.012079
column:15 min:0.000000 max:1.000000 std:0.217447
column:16 min:0.000010 max:0.500000 std:0.016379
column:17 min:0.000500 max:0.180000 std:0.007451
column:18 min:0.002500 max:0.600000 std:0.035017
column:19 min:0.017000 max:0.233000 std:0.019091
column:20 min:0.002400 max:0.6

In [185]:
### normalization

### min substr and max-min div

def normalise(df_data, cols_toNormalize):
    for col in cols_toNormalize:
        if len(col) == 1:
            print(col)
            continue
            
        min = df_data[col].min()
        max = df_data[col].max()

        if max - min == 0:
            continue

        df_data[col] = (df_data[col] - min) / (max - min)
    
    return df_data



"""
### mean substr and std dev div
for col in cols_toNormalize:
    mean = df_data[col].mean()
    std = df_data[col].std()
    
    if mean == 0 or std == 0:     ### columns 'num_outbound_cmds', 'is_host_login' has zero mean and std dev!!!
        continue
    
    df_data[col] = (df_data[col] - mean) / std
"""

"\n### mean substr and std dev div\nfor col in cols_toNormalize:\n    mean = df_data[col].mean()\n    std = df_data[col].std()\n    \n    if mean == 0 or std == 0:     ### columns 'num_outbound_cmds', 'is_host_login' has zero mean and std dev!!!\n        continue\n    \n    df_data[col] = (df_data[col] - mean) / std\n"

In [186]:
# ### Skipping normalisation for thyroid dataset. Added by Jyotirmay
df_data=normalise(df_data, cols_toNormalize)
checkStats(df_data)
print(df_data)

e
k
m
c
a
n
d
l
o
g
j
h
i
b
f
column:0 min:0.000000 max:1.000000 std:0.197341
column:1 min:0.000000 max:1.000000 std:0.461501
column:2 min:0.000000 max:1.000000 std:0.340448
column:3 min:0.000000 max:1.000000 std:0.123975
column:4 min:0.000000 max:1.000000 std:0.113844
column:5 min:0.000000 max:1.000000 std:0.192456
column:6 min:0.000000 max:1.000000 std:0.106077
column:7 min:0.000000 max:1.000000 std:0.118127
column:8 min:0.000000 max:1.000000 std:0.128450
column:9 min:0.000000 max:1.000000 std:0.242769
column:10 min:0.000000 max:1.000000 std:0.254232
column:11 min:0.000000 max:1.000000 std:0.113218
column:12 min:0.000000 max:1.000000 std:0.092386
column:13 min:0.000000 max:1.000000 std:0.157747
column:14 min:0.000000 max:1.000000 std:0.012079
column:15 min:0.000000 max:1.000000 std:0.217447
column:16 min:0.000000 max:1.000000 std:0.032758
column:17 min:0.000000 max:1.000000 std:0.041511
column:18 min:0.000000 max:1.000000 std:0.058607
column:19 min:0.000000 max:1.000000 std:0.088386


### One hot encoding of required columns

In [187]:
### obtain one hot encoding

df_data = pd.get_dummies(df_data, columns=cols_toOneHot)

- Check the resulting dataset

In [188]:
df_data.head()

,age,a,b,c,d,e,f,g,h,i,...,m,n,o,TSH,T3,TT4,T4U,FTI,label_anomaly,label_normal
4904,0.479167,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00218,0.125348,0.184937,0.430556,0.157286,0,1
1216,0.322917,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00338,0.091922,0.164854,0.291667,0.193246,0,1
3364,0.708333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00080,0.109192,0.191632,0.449074,0.157286,0,1
3286,0.072917,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.01856,0.130919,0.206695,0.365741,0.201063,0,1
1724,0.625000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00010,0.292479,0.347280,0.324074,0.373046,0,1


In [189]:
### shape is correct

df_data.shape

(6853, 23)

In [190]:
df_data.describe()

,age,a,b,c,d,e,f,g,h,i,...,m,n,o,TSH,T3,TT4,T4U,FTI,label_anomaly,label_normal
count,6853.000000,6853.000000,6853.000000,6853.000000,6853.000000,6853.000000,6853.000000,6853.000000,6853.000000,6853.000000,...,6853.000000,6853.000000,6853.000000,6853.000000,6853.000000,6853.000000,6853.000000,6853.000000,6853.000000,6853.000000
mean,0.531639,0.307603,0.133810,0.015614,0.013133,0.038523,0.011382,0.014154,0.016781,0.062892,...,0.025536,0.000146,0.049759,0.006882,0.109418,0.181021,0.373549,0.175391,0.036480,0.963520
std,0.197355,0.461535,0.340473,0.123984,0.113852,0.192470,0.106085,0.118136,0.128459,0.242787,...,0.157759,0.012080,0.217463,0.032760,0.041514,0.058611,0.088393,0.054680,0.187496,0.187496
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.375000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.001380,0.091922,0.146444,0.324074,0.146341,0.000000,1.000000
50%,0.552083,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.003180,0.109192,0.174895,0.365741,0.171357,0.000000,1.000000
75%,0.687500,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.004780,0.119777,0.206695,0.402778,0.194809,0.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [191]:
### check again

checkStats(df_data)

column:0 min:0.000000 max:1.000000 std:0.197341
column:1 min:0.000000 max:1.000000 std:0.461501
column:2 min:0.000000 max:1.000000 std:0.340448
column:3 min:0.000000 max:1.000000 std:0.123975
column:4 min:0.000000 max:1.000000 std:0.113844
column:5 min:0.000000 max:1.000000 std:0.192456
column:6 min:0.000000 max:1.000000 std:0.106077
column:7 min:0.000000 max:1.000000 std:0.118127
column:8 min:0.000000 max:1.000000 std:0.128450
column:9 min:0.000000 max:1.000000 std:0.242769
column:10 min:0.000000 max:1.000000 std:0.254232
column:11 min:0.000000 max:1.000000 std:0.113218
column:12 min:0.000000 max:1.000000 std:0.092386
column:13 min:0.000000 max:1.000000 std:0.157747
column:14 min:0.000000 max:1.000000 std:0.012079
column:15 min:0.000000 max:1.000000 std:0.217447
column:16 min:0.000000 max:1.000000 std:0.032758
column:17 min:0.000000 max:1.000000 std:0.041511
column:18 min:0.000000 max:1.000000 std:0.058607
column:19 min:0.000000 max:1.000000 std:0.088386
column:20 min:0.000000 max:1.0

In [192]:
## check integrity of the data again

print(df_data.isnull().values.any())

print(df_data.isna().values.any())

False
False


In [193]:
### check again
print(df_data.iloc[740:750, -1])

print(df_data.iloc[740:750, -2])

print(df_labels.iloc[740:750, -1])

5474    1
4216    1
2610    1
5516    1
1915    1
435     1
1095    1
4879    1
728     1
5289    1
Name: label_normal, dtype: uint8
5474    0
4216    0
2610    0
5516    0
1915    0
435     0
1095    0
4879    0
728     0
5289    0
Name: label_anomaly, dtype: uint8
5474    n
4216    n
2610    n
5516    n
1915    n
435     n
1095    n
4879    n
728     n
5289    n
Name: labels, dtype: object


### Save dataset

In [194]:
dataset = df_data.values

In [195]:
dataset.shape

(6853, 23)

In [196]:
np.savez_compressed("thyroid.npz", dataset=dataset)

### Produce train and test splits

In [197]:
random_state = None

In [198]:
from sklearn.model_selection import train_test_split

x_train, x_test = train_test_split(dataset, test_size=0.5, random_state=random_state)

In [199]:
x_train.shape

(3426, 23)

In [200]:
x_test.shape

(3427, 23)

-  Normal class is anomaly (attack) in this case, because non-attack rows are minority

In [201]:
x_train_normal = x_train[np.where(x_train[:,-1] == 1)]   ##last column is the normal column
x_train_anomaly = x_train[np.where(x_train[:,-1] == 0)]   

In [202]:
x_train_normal.shape

(3306, 23)

In [203]:
x_train_anomaly.shape   

(120, 23)

In [204]:
x_train = x_train_normal

- x_train consist of "normal" values

In [205]:
x_train.shape

(3306, 23)

In [206]:
x_test.shape

(3427, 23)

- Assign arbitrary output as labels

In [207]:
y_train = np.zeros((len(x_train),2))
y_train[:,0] = 1
y_train.shape

(3306, 2)

In [208]:
train_name = "thyroid_train-randomState_"+str(random_state)+".npz"
test_name = "thyroid_test-randomState_"+str(random_state)+".npz"

np.savez_compressed(train_name, x_train=x_train, y_train=y_train)
np.savez_compressed(test_name, x_test=x_test)